In [1]:
import os
import time
import requests
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

In [2]:
from datasets import load_dataset  # pip install datasets

ytt = load_dataset(
    "pinecone/yt-transcriptions",
    split="train",
    revision="926a45"
)

/usr/local/lib/python3.8/dist-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [3]:
for x in ytt:
    print(x)
    break

{'video_id': 'ZPewmEu7644', 'text': " hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you", 'start_second': 0, 'end_second': 20, 'url': 'https://www.youtube.com/watch?v=ZPewmEu7644&t=0s', 'title': 'GANS for Semi-Supervised Learning in Keras (7.4)', 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}


In [4]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

retriever = SentenceTransformer(
    'flax-sentence-embeddings/all_datasets_v3_mpnet-base'
)
retriever.to(device)

2024-01-20 06:25:17.447654: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-20 06:25:17.496286: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-20 06:25:17.497847: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-20 06:25:18.334668: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [5]:
embed_dim = retriever.get_sentence_embedding_dimension()
embed_dim

768

In [6]:
from pinecone import Pinecone, ServerlessSpec

# get api key from app.pinecone.io
pinecone = Pinecone(
    api_key="55e1c7a7-6374-4a46-9650-b075f74e2158",
    environment="us-west1-gcp"
)

# connect to new index
index = pinecone.Index("youtube-search")

In [7]:
from tqdm.auto import tqdm

docs = []  # this will store IDs, embeddings, and metadata

batch_size = 64

for i in tqdm(range(0, len(ytt), batch_size)):
    i_end = min(i+batch_size, len(ytt))
    # extract batch from YT transactions data
    batch = ytt[i:i_end]
    # encode batch of text
    embeds = retriever.encode(batch['text']).tolist()
    # each snippet needs a unique ID
    # we will merge video ID and start_seconds for this
    ids = [f"{x[0]}-{x[1]}" for x in zip(batch['video_id'], batch['start_second'])]
    # create metadata records
    meta = [{
        'video_id': x[0],
        'title': x[1],
        'text': x[2],
        'start_second': x[3],
        'end_second': x[4],
        'url': x[5],
        'thumbnail': x[6]
    } for x in zip(
        batch['video_id'],
        batch['title'],
        batch['text'],
        batch['start_second'],
        batch['end_second'],
        batch['url'],
        batch['thumbnail']
    )]
    # create list of (IDs, vectors, metadata) to upsert
    to_upsert = list(zip(ids, embeds, meta))
    # add to pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/177 [00:00<?, ?it/s]

In [17]:
query = "Teach me transformers"

xq = retriever.encode([query]).tolist()

In [18]:
xc = index.query(vector=xq, top_k=5,
                 include_metadata=True)
# for context in xc['matches']:
#     print(context['metadata']['text'], end="\n---\n")

In [19]:
xc

{'matches': [{'id': 'Uumd2zOOz60-114',
              'metadata': {'end_second': 139.0,
                           'start_second': 114.0,
                           'text': ' a video if made lots of videos on '
                                   'transformers attention is all you need is '
                                   'the base paper for that so I know what a '
                                   'transformer is okay and I know that '
                                   'transformers are usually in NLP are '
                                   'usually used in NLP door there are things '
                                   'like you know other things with '
                                   'transformers but usually an NLP model then '
                                   'I read object detection and I know object '
                                   'detection is',
                           'thumbnail': 'https://i.ytimg.com/vi/Uumd2zOOz60/maxresdefault.jpg',
                         

In [ ]:
# S